**This notebook will do the following:**
- Create the required folders in the File section
- Bind the cicd notebooks to the cicd lakehouse

**Manual step**
- Manually create a schema enabled cicd lakehouse 
- Remove the existing attached lakehouse and attach the newly created cicd lakehouse to this notebook

**Variables**

In [ ]:
vLakehouseName = "cicdlakehouse" # the name of your lakehouse
vCicdFolderName = "cicd"
vConnectionFolderName = "connections" # folder to host the extraction of exiting connections 
vOnelakeAccessFolderName = "onelake_access"
vProjectName = "fabric-cicd" # replace by your ci
vDebugMode = "yes"

**Helper notebook**

In [ ]:
%run nb_helper

**Token and base url**

In [ ]:
vApiVersion = "v1"
vScope = "https://analysis.windows.net/powerbi/api"
vAccessToken  = mssparkutils.credentials.getToken(vScope)
vBaseUrl = f"https://api.fabric.microsoft.com/{vApiVersion}/"
vHeaders = {'Authorization': f'Bearer {vAccessToken}'}

**Resolve current workspace name and id**

In [ ]:
vWorkspaceName, vWorkspaceId = fabric.resolve_workspace_name_and_id()
print(vWorkspaceName, vWorkspaceId)

**Create a schema called staging**

In [ ]:
%%sql
CREATE SCHEMA staging

**Create the folders in the lakehouse**

In [ ]:
# this folder can be used to host a file containing the list of existing connections in a fabric tenant
# the list can help creating the mapping_connections.json secure file required for running post deployment steps in the yaml pipeline
# it is not specific to a particular project
notebookutils.fs.mkdirs(f"Files/{vCicdFolderName}/{vConnectionFolderName}")  

# this folder can be use to host the onelake access files generated by nb_extract_lakehouse_access
# these files can help the creation of dedicated roles in target lakehouses 
# the folder should be specific to a project
notebookutils.fs.mkdirs(f"Files/{vCicdFolderName}/{vOnelakeAccessFolderName}/{vProjectName}") 

**Manual step**
- For all projects --> extract the list of connections in the tenant
- Per project --> run nb_extract_lakehouse_access to extract the onelake roles definition for a specific project

**Define the default lakehouse in the cicd notebooks:**
- nb_cicd_post_deployment
- nb_cicd_post_update_data_pipelines
- nb_cicd_post_update_notebooks
- nb_cicd_post_update_semantic_models
- nb_cicd_pre_deployment
- nb_cicd_pre_update_lakehouses
- nb_cicd_pre_update_warehouses

In [ ]:
vLakehouseId = fabric.resolve_item_id(vLakehouseName)

# get the list of data pipelines in the target workspace
vNotebookList = [
    'nb_cicd_post_deployment',
    'nb_cicd_post_update_data_pipelines',
    'nb_cicd_post_update_notebooks',
    'nb_cicd_post_update_semantic_models',
    'nb_cicd_pre_deployment',
    'nb_cicd_pre_update_lakehouses',
    'nb_cicd_pre_update_warehouses'
]

df_notebooks = notebookutils.notebook.list(workspaceId=vWorkspaceId)
for notebook in df_notebooks:
    
    # get the notebook id and display name
    vNotebookId = notebook.id
    vNotebookName = notebook.displayName

    if vNotebookName in vNotebookList:

        # get the current notebook definition
        vNotebookDefinition = notebookutils.notebook.getDefinition(name=vNotebookName, workspaceId=vWorkspaceId) 
        vNotebookJson = json.loads(vNotebookDefinition)

        # update lakehouse dependencies
        try:

            # check and remove any attached lakehouses
            if 'dependencies' in vNotebookJson['metadata'] \
                and 'lakehouse' in vNotebookJson['metadata']['dependencies'] \
                and vNotebookJson['metadata']["dependencies"]["lakehouse"] is not None:

                vCurrentLakehouse = vNotebookJson['metadata']['dependencies']['lakehouse']
                # print(vCurrentLakehouse)

                if 'default_lakehouse_name' in vCurrentLakehouse:

                    vNotebookJson['metadata']['dependencies']['lakehouse'] = {}
                    print(f"attempting to update notebook <{vNotebookName}> with new default lakehouse: {vCurrentLakehouse['default_lakehouse_name']} in workspace <{vWorkspaceName}>.")

                    # update new notebook definition after removing existing lakehouses and with new default lakehouseId
                    notebookutils.notebook.updateDefinition(
                        name = vNotebookName,
                        content  = json.dumps(vNotebookJson),  
                        defaultLakehouse = vLakehouseName, #vCurrentLakehouse['default_lakehouse_name'],
                        defaultLakehouseWorkspace = vWorkspaceId,
                        workspaceId = vWorkspaceId
                    )

                    print(f"updated notebook <{vNotebookName}> in workspace <{vWorkspaceName}>.")

                else:
                    print(f'no default lakehouse set for notebook <{vNotebookName}>, ignoring.')

            vMessage = f"succeeded"
        except Exception as e:
            vMessage = f"failed"
            if vDebugMode == "yes":
                print(str(e))